In [1]:
%matplotlib
import numpy as np
import equations as equ
from scipy.optimize import fsolve
import math
from  librerieTesi.diffuseRaman import core
import matplotlib.pyplot as plt
from scipy.optimize import leastsq
import core_functions as cf
from scipy.special import iv
import montecarlomodule as mc
v = 29.97925/1.4
pi = math.pi
exp =np.exp
log = np.log    
sqrt = np.sqrt
mu_s = 10

Using matplotlib backend: Qt5Agg


caso $\mu_a$ = 0

In [2]:
def te_c(t, rho, mu_s):
    #C = (mu_s/3)**(3/2)/(8*math.pi*v)
    #z_e = 2*z_e
    C = (v/(8*pi))*(3*mu_s**2)**(3/2)
    x = (t*v*mu_s)**2-3*rho**2*mu_s**2
    
    set_zero = x<=0
    x = sqrt(x)
    res = C*iv(1,x/2) * np.exp(-t*v*mu_s/2)/x
    res[set_zero] = 0
    use_exp = (x/2)> 100
    res_exp = te_1(t, rho, mu_s)
    res[use_exp] = res_exp[use_exp]
    return res
def te_1(t,rho,  mu_s):
    #z_e = 2*z_e
    C = (v/(8*pi))*(3*mu_s**2)**(3/2)
    x = (t*v*mu_s)**2-3*rho**2*mu_s**2 
    x = sqrt(x)
    mbf = lambda x , x2 : exp(x-x2)/sqrt(2*pi*x)
    return  C*mbf(x/2, t*v*mu_s/2)/x

In [3]:
import latexify

In [4]:
@latexify.function

def diff_eq(t, r, mu_s):
    D = 1/(3*mu_s)
    res = v/(4*pi*D*v*t)**(3/2)*exp(-r**2/(4*D*v*t))
    if t[0] == 0:
        res[0] = 0
    return res
diff_eq

In [5]:
@latexify.function
def diff_eq_corrEXP(t, r, mu_s):
    de = diff_eq(t, r, mu_s)
    conv_mat = core.conv_matrix(np.exp(-mu_s*v*t), len(t))
    return np.matmul(conv_mat, de)
diff_eq_corrEXP

In [6]:
@latexify.function
def diff_eq_corrDER(t, r, mu_s):
    de = diff_eq(t, r, mu_s)
    d_de_dt = cf.derivative(de, x)
    return de - 1/(mu_s*v)*d_de_dt
diff_eq_corrDER

In [7]:
@latexify.function
def te_c(t, rho, mu_s):
    #C = (mu_s/3)**(3/2)/(8*math.pi*v)
    #z_e = 2*z_e
    C = (v/(8*pi))*(3*mu_s**2)**(3/2)
    x = (t*v*mu_s)**2-3*(rho**2)*mu_s**2
    x = sqrt(x)
    res = C*iv(1,sqrt((t*v*mu_s)**2-3*(rho**2)*mu_s**2)/2) * np.exp(-t*v*mu_s/2)/sqrt((t*v*mu_s)**2-3*(rho**2)*mu_s**2)
    return res
te_c

In [8]:
def te_M(t, rho, mu_s):
    C = v*(3*mu_s)**3/(8*pi)
    
    x = (t*v*3*mu_s)**2-(rho**2**2)*(3*mu_s)**2
    #plt.plot(sqrt(x)/2-t*v*(3*mu_s)/2)
    if np.isscalar(x):
        if x<=0:
            return 0     
        x = sqrt(x)
        if (x/2)> 100    :
            return te_M_1(t, rho, mu_s)
        res = C*iv(1,x/2) * np.exp(-t*v*3*mu_s/2)/x
        return res
    set_zero = x<=0
    x = sqrt(x)
    res = C*iv(1,x/2) * np.exp(-t*v*3*mu_s/2)/x
    res[set_zero] = 0
    use_exp = (x/2)> 100
    res_exp = te_M_1(t, rho, mu_s)
    res[use_exp] = res_exp[use_exp]
    return res
def te_M_1(t,rho,  mu_s):
    #z_e = 2*z_e
    C = v*(3*mu_s)**3/(8*pi**(3/2))
    x = (t*v*3*mu_s)**2-rho**2*(3*mu_s)**2 
    #mbf = lambda x , x2 : exp(x-x2)/sqrt(2*pi*x)
    return  C*exp(sqrt(x)/2-(3*mu_s*v*t)/2)/x**(3/4)#C* mbf(x/2, t*v*3*mu_s/2)/x

In [9]:
class obj_det:
    def __init__(self, x,y,z, r):
        self.x = x
        self.y = y
        self.z = z
        self.r = r
d = obj_det(0,0,0,0.1)

In [ ]:
for mu_s in [1, 5, 10, 20, 40, 80]:
    res = mc.mc(0.5,mu_s, d)
    time = np.arange(0,len(res))/1e3
    plt.plot(time, res/np.sum(res[100:]), label = "$\mu_{s,mc}$" +str(mu_s))
    res = diff_eq_corrEXP(time, 0, mu_s*(1-0.5))
    plt.plot(time, res/np.sum(res[100:]), label = "$\mu_{s,exp}$" +str(mu_s))
    res = diff_eq(time, 0, mu_s*(1-0.5))
    plt.plot(time, res/np.sum(res[100:]), label = "$\mu_{s,de}$" +str(mu_s))
    res = diff_eq_corrDER(time, 0, mu_s*(1-0.5))
    plt.plot(time, res/np.sum(res[100:]), label = "$\mu_{s,der}$" +str(mu_s))


In [195]:
for mu_s in [5,10, 15,20]:
    def derivative(x,y):
        dx = x[1]-x[0]
        return np.gradient(y, dx)
    d_de_dt = derivative(time, res)
    res_d_dt = res+1/(mu_s*v)*d_de_dt
    t_min = 0
    for i in range(len(res_d_dt)):
        if res_d_dt[i]<0:
            t_min = i
    sub_zero = time<=time[t_min]
    res_d_dt[sub_zero] = 0
    plt.plot(time,res_d_dt/np.sum(res_d_dt), label = str(mu_s))
plt.yscale('log')
plt.legend()

In [176]:
def derivative(x,y):
    dx = x[1]-x[0]
    return np.gradient(y, dx)
d_de_dt = derivative(time, res)
res_d_dt = res-1/(mu_s*v)*d_de_dt
sub_zero = res_d_dt<0
res_d_dt[sub_zero] = 0
conv_mat = core.conv_matrix(exp(-mu_s*v*time), len(time))
res_exp = np.matmul(conv_mat, res)

In [194]:
for mu_s in [5,10, 15,20]:
    def derivative(x,y):
        dx = x[1]-x[0]
        return np.gradient(y, dx)
    d_de_dt = derivative(time, res)
    res_d_dt = res-1/(mu_s*v)*d_de_dt
    sub_zero = res_d_dt<0
    res_d_dt[sub_zero] = 0
    conv_mat = core.conv_matrix(exp(-mu_s*v*time), len(time))
    res_exp = np.matmul(conv_mat, res)
    te = te_c(time, 0, mu_s)
    plt.plot(time,te/np.sum(te), label = str(mu_s))
plt.yscale('log')
plt.legend()

/tmp/ipykernel_5256/2292413621.py:9: RuntimeWarning: invalid value encountered in true_divide
  res = C*iv(1,x/2) * np.exp(-t*v*mu_s/2)/x
/tmp/ipykernel_5256/2292413621.py:20: RuntimeWarning: divide by zero encountered in true_divide
  mbf = lambda x , x2 : exp(x-x2)/sqrt(2*pi*x)
/tmp/ipykernel_5256/2292413621.py:9: RuntimeWarning: invalid value encountered in true_divide
  res = C*iv(1,x/2) * np.exp(-t*v*mu_s/2)/x
/tmp/ipykernel_5256/2292413621.py:20: RuntimeWarning: divide by zero encountered in true_divide
  mbf = lambda x , x2 : exp(x-x2)/sqrt(2*pi*x)
/tmp/ipykernel_5256/2292413621.py:9: RuntimeWarning: invalid value encountered in true_divide
  res = C*iv(1,x/2) * np.exp(-t*v*mu_s/2)/x
/tmp/ipykernel_5256/2292413621.py:20: RuntimeWarning: divide by zero encountered in true_divide
  mbf = lambda x , x2 : exp(x-x2)/sqrt(2*pi*x)
/tmp/ipykernel_5256/2292413621.py:9: RuntimeWarning: invalid value encountered in true_divide
  res = C*iv(1,x/2) * np.exp(-t*v*mu_s/2)/x
/tmp/ipykernel_5256

In [177]:
for b in [1]:
    #plt.figure()
    #plt.plot(time,res/np.sum(res[b:]), label = "de10")
    plt.plot(time,res_d_dt/np.sum(res_d_dt[b:]), label = "corr10")
    plt.plot(time,data/np.sum(data[b:]), label = "data10")
    #plt.plot(time,res_exp/np.sum(res_exp[b:]), label = "exp")
    plt.yscale('log')
    #plt.xlim(left = -0.01, right = 0.1)
    #plt.ylim(bottom = 1e-2)
    plt.legend()
    #plt.show()

In [117]:
plt.figure()
plt.yscale('log')
plt.xlim(left = -1,right = 50)
plt.ylim(bottom = 1e-2)
plt.scatter(np.arange(1000),1-np.cumsum(res)/np.sum(res))


In [81]:
def linsquare(data,simulation):
    ls = []
    for i in range(100):
        ls.append(np.linalg.norm(data[i:]/np.sum(data[i:])-simulation[i:]/np.sum(simulation[i:])))
    return np.array(ls)
    

In [119]:
plt.plot(linsquare(data,res), label = "de")
plt.plot(linsquare(data,res_d_dt), label = "derivative")
plt.plot(linsquare(data,res_exp), label = "exp")
plt.yscale("log")
plt.legend()


fare convoluzione in r

In [49]:
b = 2
plt.plot(time,res/np.sum(res[b:]), label = "de")
plt.show()